In [1]:
import numpy as np
from pyqubo import Binary
import pyqubo
import neal
import matplotlib.pyplot as plt 
from sympy import *
from datetime import datetime
import pandas as pd

## Input Data 

In [95]:
# Input data

#Assets
asset_quantity = pd.read_csv("data/sample_asset_quantity.csv", header=None)[0].to_numpy()
asset_value = pd.read_csv("data/sample_asset_value.csv", header=None)[0].to_numpy()
asset_tiers = pd.read_csv("data/sample_asset_tiers.csv", header=None)[0].to_numpy()

n_assets = len(asset_quantity)


#Accounts 
account_exposure = pd.read_csv("data/sample_account_exposure.csv", header=None)[0].to_numpy()
account_duration = pd.read_csv("data/sample_account_duration.csv", header=None)[0].to_numpy()

n_accounts = len(account_exposure)

#single limits
single_limits = pd.read_csv("data/sample_single_limits.csv", header=None).to_numpy() 

#haircuts 
haircuts = pd.read_csv("data/sample_haircuts.csv", header=None).to_numpy()/100


#Updated Tier list 

cost_factor_matrix = np.zeros(shape=(n_assets, n_accounts))

for i in range(n_assets):
    for j in range(n_accounts):
        cost_factor_matrix[i,j] = abs(account_duration[j] - asset_tiers[i])


# Penalty Terms


In [135]:
# Penalty Terms
lambda_cost_fn = 1e4
lambda_consistency1, lambda_consistency2 = 1, 1
lambda_exposure1, lambda_exposure2 =1, 3e1


# Constructing the QUBO

In [136]:
binary_variables = []
n_binary = 7
max_binary_value = 2**n_binary - 1

single_limits_fractional = np.zeros_like(single_limits) 
single_limits_num_bits = np.zeros((n_assets,n_accounts), dtype=int) 

for i in range(n_assets):
    for j in range(n_accounts):
        single_limits_fractional[i][j] = single_limits[i][j]/asset_quantity[i]
        aux_variable = int(np.floor(np.log2((single_limits_fractional[i][j])*max_binary_value)))
        single_limits_num_bits[i][j] = n_binary if aux_variable > n_binary else aux_variable
        
        for k in range(single_limits_num_bits[i][j]):
            globals()["x" + str(i + 1) + str(j + 1) + str(k)] = Binary("x" + str(i + 1) + str(j + 1) + str(k))
            binary_variables.append(globals()["x" + str(i + 1) + str(j + 1) + str(k)])



In [137]:

cost_function = 0

scale_cost_fn = 1000

for i in range(n_assets):
    for j in range(n_accounts):
        aux_term = 0
        coeff = (cost_factor_matrix[i][j])/ (max_binary_value)
        for k in range(single_limits_num_bits[i][j]):
            aux_term += (2 ** (k % n_binary)) * globals()["x" + str(i + 1) + str(j + 1) + str(k)] 
        cost_function += scale_cost_fn*lambda_cost_fn*coeff*aux_term
        

In [138]:
con_constraint = 0
scale_consis_1 = 1000
scale_consis_2 = 100000

for i in range(n_assets):
    aux_term1 = 0
    aux_term2 = 0
    for j in range(n_accounts):
        for k in range(single_limits_num_bits[i][j]):
            aux_term1 += (2 ** (k % n_binary)) * globals()["x" + str(i + 1) + str(j + 1) + str(k)]
    aux_term2 = (aux_term1/(max_binary_value)) - 1
    con_constraint += scale_consis_1*(lambda_consistency1 * aux_term2) + scale_consis_2*(lambda_consistency2 * (aux_term2**2))


In [139]:

exposure_term1 = 0
exposure_term2 = 0
for j in range(n_accounts):
    aux_term2 = 0
    aux_term3 = 0
    for i in range(n_assets):
        aux_term1 = 0
        for k in range(single_limits_num_bits[i][j]):
            aux_term1 += (2 ** (k % n_binary)) * globals()["x" + str(i + 1) + str(j + 1) + str(k)]
        aux_term2 += (aux_term1/max_binary_value) * asset_quantity[i] * asset_value[i] * haircuts[i][j]
    aux_term3 += aux_term2 - account_exposure[j]
    exposure_term1 += -lambda_exposure1 * aux_term3
    exposure_term2 += lambda_exposure2 * (aux_term3 ** 2)


norm_exposure = max(asset_quantity*asset_value)*64/max_binary_value
scale_exposure_1 = 1e4
scale_exposure_2 = 1e7

exposure_requirements = scale_exposure_1*(exposure_term1/norm_exposure)  + scale_exposure_2*(exposure_term2/norm_exposure**2)


In [140]:
# Full QUBO

final_equation = cost_function + con_constraint + exposure_requirements 

# Compile QUBO
model = final_equation.compile()
# Obtain the Q matrix
qubo, offset = model.to_qubo()

# Run Solver

In [141]:
start=datetime.now()

# Use solver to find solution
bqm = model.to_bqm()
sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, num_reads=100, num_sweeps = 1000)
decoded_samples = model.decode_sampleset(sampleset)

# Get results
best_sample = min(decoded_samples, key=lambda x: x.energy)
results = best_sample.sample
print(f"The minimum energy is: {best_sample.energy}")

The minimum energy is: 6330051.708519697


# Verify Results

In [148]:


exposure_solution = np.zeros(shape = (n_accounts))
allocation_solution = np.zeros(shape = (n_assets))

print("=====Each Individual Allocation Percentage=====")

for i in range(n_assets):
    asset = 0
    for j in range(n_accounts):
        decimal = 0
        for k in range(single_limits_num_bits[i][j]):
            decimal +=(2**k)*results.get("x" + str(i + 1) + str(j + 1) + str(k))
        asset += decimal*(100/max_binary_value)
        print(f"Assign {decimal*(100/max_binary_value):0.2f}% of asset {i+1} to account {j+1}")
        exposure_solution[j] += (decimal /max_binary_value) * asset_value[i] * asset_quantity[i]*haircuts[i][j]
    allocation_solution[i] = asset


=====Each Individual Allocation Percentage=====
Assign 0.00% of asset 1 to account 1
Assign 0.00% of asset 1 to account 2
Assign 0.00% of asset 1 to account 3
Assign 0.00% of asset 1 to account 4
Assign 0.00% of asset 1 to account 5
Assign 37.01% of asset 2 to account 1
Assign 11.81% of asset 2 to account 2
Assign 0.00% of asset 2 to account 3
Assign 0.00% of asset 2 to account 4
Assign 0.00% of asset 2 to account 5
Assign 0.00% of asset 3 to account 1
Assign 0.00% of asset 3 to account 2
Assign 0.00% of asset 3 to account 3
Assign 0.00% of asset 3 to account 4
Assign 0.00% of asset 3 to account 5
Assign 11.81% of asset 4 to account 1
Assign 23.62% of asset 4 to account 2
Assign 0.00% of asset 4 to account 3
Assign 0.00% of asset 4 to account 4
Assign 0.00% of asset 4 to account 5
Assign 11.81% of asset 5 to account 1
Assign 11.81% of asset 5 to account 2
Assign 20.47% of asset 5 to account 3
Assign 0.79% of asset 5 to account 4
Assign 5.51% of asset 5 to account 5
Assign 0.00% of asse

In [143]:
print("====Validation for Consistency====")
for i in range(n_assets):
    print(f"{allocation_solution[i]:0.2f}% of asset {i+1} posted")

====Validation for Consistency====
0.00% of asset 1 posted
48.82% of asset 2 posted
0.00% of asset 3 posted
35.43% of asset 4 posted
50.39% of asset 5 posted
6.30% of asset 6 posted
0.00% of asset 7 posted
51.97% of asset 8 posted
0.00% of asset 9 posted
0.00% of asset 10 posted


In [144]:
print("=====Validation for Exposure======")
for i in range(n_accounts):
    print(f"${exposure_solution[i]:,} of collateral posted, ${account_exposure[i]} required")
    print(f"{(exposure_solution[i]/account_exposure[i] - 1) *100.:0.2f}%")

=====Validation for Exposure======
$814,414.2998264052 of collateral posted, $824000.0 required
-1.16%
$655,733.9563175056 of collateral posted, $669292.0 required
-2.03%
$538,647.9315566496 of collateral posted, $548600.0 required
-1.81%
$56,953.30570639067 of collateral posted, $64844.52 required
-12.17%
$465,006.63375560043 of collateral posted, $472000.0 required
-1.48%


In [145]:
print("===Validation of Single limits =====")
for i in range(n_assets):
    for j in range(n_accounts):
        decimal = 0
        for k in range(single_limits_num_bits[i][j]):
            decimal +=(2**k)*results.get("x" + str(i + 1) + str(j + 1) + str(k))
        allocation = (decimal/max_binary_value)*asset_quantity[i] 
        print(f"Amount allocated: {allocation:0.2f}. Single Limit: {single_limits[i][j]:0.2f}")
        print(f"Constraint Satisfied?: {allocation <= single_limits[i][j]}")
        

===Validation of Single limits =====
Amount allocated: 0.00. Single Limit: 20997.72
Constraint Satisfied?: True
Amount allocated: 0.00. Single Limit: 153498.10
Constraint Satisfied?: True
Amount allocated: 0.00. Single Limit: 27959.58
Constraint Satisfied?: True
Amount allocated: 0.00. Single Limit: 4957.23
Constraint Satisfied?: True
Amount allocated: 0.00. Single Limit: 12027.82
Constraint Satisfied?: True
Amount allocated: 83447.20. Single Limit: 225485.00
Constraint Satisfied?: True
Amount allocated: 26632.09. Single Limit: 47932.09
Constraint Satisfied?: True
Amount allocated: 0.00. Single Limit: 8730.80
Constraint Satisfied?: True
Amount allocated: 0.00. Single Limit: 5159.90
Constraint Satisfied?: True
Amount allocated: 0.00. Single Limit: 225485.00
Constraint Satisfied?: True
Amount allocated: 0.00. Single Limit: 634872.00
Constraint Satisfied?: True
Amount allocated: 0.00. Single Limit: 789538.89
Constraint Satisfied?: True
Amount allocated: 0.00. Single Limit: 143814.01
Const

In [146]:
print(datetime.now()-start)

0:00:05.469046


# Export to .csv

In [147]:
# Saving to an array
Q_value_unbalanced_py_qubo = np.zeros(shape=(n_assets,n_accounts))

for i in range(n_assets):
    asset = 0
    for j in range(n_accounts):
        bit = ''
        decimal = 0
        for k in range(single_limits_num_bits[i][j]):
            bit += str(results.get("x" + str(i + 1) + str(j + 1) + str(k)))
            decimal +=(2**k)*results.get("x" + str(i + 1) + str(j + 1) + str(k))
        asset += decimal*(100/max_binary_value)
        Q_value_unbalanced_py_qubo[i][j] = decimal/max_binary_value

pprint(f"Objective value: {sum(sum(Q_value_unbalanced_py_qubo*cost_factor_matrix))}")

Objective value: 0.5559055118110235


In [ ]:
"""Uncomment the last line when you are happy with the results"""

#pd.DataFrame(Q_value_unbalanced_py_qubo).to_csv("Q_value_unbalanced_py_qubo_no_single_lim1.csv", header=None, index=None)
